In [55]:
import csv

dat_folder = '/Users/lorenzobacchiani/Desktop/birex_orchestrator/'
files = ['edge.log','cloud.log']
header = ['time', 'latency', 'size']


for file in files:
    i = 0
    time = 0
    data = []
    with open(dat_folder + file, "r") as to_read:
        for l in to_read.readlines():
            if '------' in l:
                with open(dat_folder + file.split('.')[0] + "_csv/" + "run_" + file.split('.')[0] + "_" + str(i), 'w', encoding='UTF8') as to_write:
                    writer = csv.writer(to_write)
                    writer.writerow(header)
                    for d in data:
                        writer.writerow(d)
                i = i + 1
                data = []
                time = 0
            else:
                x = l[:-2].split("(")[-1].split(",")
                lat, by = float(x[0]), float(x[1]) / 10**6
                data.append([time,lat,by])
                time = time + 10

In [91]:
import pandas as pd
import os 

scenarios = ['edge_csv/', 'cloud_csv/']

column_names = [
    "Time (sec)",
    "Latency (ms)",
    "Size (MB)"
]

usecols = [0,1,2]
dfs_dict = {}

for s in scenarios:
    dfs_list = []
    for file in os.listdir(dat_folder + s):
        if not os.path.isdir(dat_folder + s + file):
            df = pd.read_csv(
                dat_folder + s +file, 
                sep=',',
                usecols=usecols,
                names=column_names,
                skiprows=[0])
            dfs_list += [df]
    dfs_dict[s[:-1]] = dfs_list

In [92]:
dfs = {}
for s in scenarios:
    dfs_groupby = pd.concat(dfs_dict[s[:-1]]).groupby('Time (sec)')
    dfs[s[:-1]] = (dfs_groupby.mean(), dfs_groupby.std(), dfs_groupby.median())
dfs['cloud_csv'][1].reset_index()

,Time (sec),Latency (ms),Size (MB)
0,0,4157.789706,7.816322
1,10,4170.137416,7.343690
2,20,4334.995016,6.179645
3,30,3888.576177,5.270539
4,40,3710.930370,4.865869
...,...,...,...
115,1150,2493.126433,1.505041
116,1160,3132.268228,1.478645
117,1170,3278.925615,1.377051
118,1180,3366.426520,1.914913


In [111]:
import numpy as np

scenario = 'edge_csv'
col = 'Latency (ms)'

dfs = dfs_dict[scenario]
flier_high = np.empty(120) #massimo
flier_low = np.empty(120) #minimo
center = np.empty(120) #mediana
variance = np.empty(120) #varianza
for time in range(0,120,1):
    np.append(flier_high, pd.concat(dfs)[col][time].max())
    np.append(flier_low,pd.concat(dfs)[col][time].min())
    np.append(center, pd.concat(dfs)[col][time].median())
    np.append(variance,pd.concat(dfs)[col][time].var())


120


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

